In [ ]:
import io
import os
import re
import shutil
import string
import tensorflow as tf
from datetime import datetime
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [ ]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
dataset=tf.keras.utils.get_file("aclImdb_v1.tar.gz",url,untar=True,cache_dir='.',cache_subdir='')
dataset_dir=os.path.join(os.path.dirname(dataset),'aclImdb')

84131840/84125825 [==============================] - 2s 0us/step


In [ ]:
current_dir=os.getcwd()
os.listdir(current_dir)

['.config', 'aclImdb', 'aclImdb_v1.tar.gz.tar.gz', 'sample_data']

In [ ]:
dataset_dir=os.path.join(current_dir,'aclImdb')
dataset_dir

'/content/aclImdb'

In [ ]:
os.listdir(dataset_dir)

['train', 'test', 'imdbEr.txt', 'README', 'imdb.vocab']

In [ ]:
train_dir=os.path.join(dataset_dir,'train')
os.listdir(train_dir)

['unsupBow.feat',
 'labeledBow.feat',
 'neg',
 'urls_neg.txt',
 'pos',
 'unsup',
 'urls_pos.txt',
 'urls_unsup.txt']

In [ ]:
remove_dir=os.path.join(train_dir,'unsup')
shutil.rmtree(remove_dir)

In [ ]:
os.listdir(train_dir)

['unsupBow.feat',
 'labeledBow.feat',
 'neg',
 'urls_neg.txt',
 'pos',
 'urls_pos.txt',
 'urls_unsup.txt']

In [ ]:
batch_size = 1024
seed=123
train_dataset=tf.keras.preprocessing.text_dataset_from_directory('aclImdb/train',
                                                                batch_size=batch_size,subset="training",
                                                                validation_split=0.2,seed=123)
validation_dataset=tf.keras.preprocessing.text_dataset_from_directory('aclImdb/train',batch_size=batch_size,
                                                                     subset="validation",seed=123,validation_split=0.2)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [ ]:
for text_batch,label_batch in train_dataset.take(1):
    for i in range(5):
        print(label_batch[i].numpy(),text_batch.numpy()[i])
        print("-----------------------------------")

0 b"Oh My God! Please, for the love of all that is holy, Do Not Watch This Movie! It it 82 minutes of my life I will never get back. Sure, I could have stopped watching half way through. But I thought it might get better. It Didn't. Anyone who actually enjoyed this movie is one seriously sick and twisted individual. No wonder us Australians/New Zealanders have a terrible reputation when it comes to making movies. Everything about this movie is horrible, from the acting to the editing. I don't even normally write reviews on here, but in this case I'll make an exception. I only wish someone had of warned me before I hired this catastrophe"
-----------------------------------
1 b'This movie is SOOOO funny!!! The acting is WONDERFUL, the Ramones are sexy, the jokes are subtle, and the plot is just what every high schooler dreams of doing to his/her school. I absolutely loved the soundtrack as well as the carefully placed cynicism. If you like monty python, You will love this film. This mov

In [ ]:
tf.data.experimental.AUTOTUNE

-1

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
embedding_layer=tf.keras.layers.Embedding(1000,5)
result=embedding_layer(tf.constant([1,2,3]))
result.numpy()

array([[ 0.0486437 ,  0.04697   ,  0.02553776, -0.0122981 ,  0.00291988],
       [-0.00820967, -0.04513201, -0.03722973, -0.04038912, -0.04568353],
       [-0.00605314,  0.03646765,  0.00811279,  0.00334601,  0.02905396]],
      dtype=float32)

In [ ]:
result = embedding_layer(tf.constant([[0,1,2],[3,4,5]]))
result.shape

TensorShape([2, 3, 5])

In [ ]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html,
                                      '[%s]' % re.escape(string.punctuation), '')

In [ ]:
vocab_size = 10000
sequence_length = 100

In [ ]:
vectorize_layer = TextVectorization(
                    standardize=custom_standardization,
                    max_tokens=vocab_size,
                    output_mode='int',
                    output_sequence_length=sequence_length)

In [ ]:
text_ds=train_dataset.map(lambda x,y:x)
vectorize_layer.adapt(text_ds)

In [ ]:
embedding_dim=16
model = Sequential([
                  vectorize_layer,
                  Embedding(vocab_size, embedding_dim, name="embedding"),
                  GlobalAveragePooling1D(),
                  Dense(16, activation='relu'),
                  Dense(1)])

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(train_dataset,
          validation_data=validation_dataset, 
          epochs=15,
          callbacks=[tensorboard_callback])

Epoch 1/15
20/20 [==============================] - 4s 152ms/step - loss: 0.6929 - accuracy: 0.5037 - val_loss: 0.6909 - val_accuracy: 0.4886
Epoch 2/15
20/20 [==============================] - 1s 75ms/step - loss: 0.6897 - accuracy: 0.5037 - val_loss: 0.6854 - val_accuracy: 0.4886
Epoch 3/15
20/20 [==============================] - 2s 76ms/step - loss: 0.6830 - accuracy: 0.5037 - val_loss: 0.6753 - val_accuracy: 0.4886
Epoch 4/15
20/20 [==============================] - 2s 76ms/step - loss: 0.6711 - accuracy: 0.5037 - val_loss: 0.6593 - val_accuracy: 0.4886
Epoch 5/15
20/20 [==============================] - 2s 76ms/step - loss: 0.6525 - accuracy: 0.5038 - val_loss: 0.6369 - val_accuracy: 0.4916
Epoch 6/15
20/20 [==============================] - 1s 75ms/step - loss: 0.6266 - accuracy: 0.5241 - val_loss: 0.6089 - val_accuracy: 0.5610
Epoch 7/15
20/20 [==============================] - 1s 75ms/step - loss: 0.5944 - accuracy: 0.6024 - val_loss: 0.5775 - val_accuracy: 0.6348
Epoch 8/15
2

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 100)               0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [ ]:
weights=model.get_layer('embedding').get_weights()[0]
vocab=vectorize_layer.get_vocabulary()

In [ ]:
len(vocab)

10000

In [ ]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

In [ ]:
for index, word in enumerate(vocab):
   if index == 0: continue
   vec = weights[index]
   out_v.write('\t'.join([str(x) for x in vec]) + "\n")
   out_m.write(word + "\n")
out_v.close()
out_m.close()

In [ ]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception as e:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>